In [37]:
import pandas as pd
import numpy as np

In [39]:
sample = pd.read_csv('./small_sample.csv')

In [3]:
df = pd.concat([sample,sample,sample,sample,sample,sample],axis = 0)

In [4]:
def devide(t,h):
    if h == 0:
        return np.nan
    return t/h

In [20]:
%%timeit
iterrows_devide = []
for index,row in df.iterrows():
    Time = row['Time']
    HelpfulnessNumerator = row['HelpfulnessNumerator']
    iterrows_devide.append(devide(Time,HelpfulnessNumerator))

5min 12s ± 31.1 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
%%timeit
df.apply(lambda row: devide(row['Time'],row['HelpfulnessNumerator']),axis = 1)

1min 25s ± 4.88 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
%%timeit
itertuples_devide = []
for _ in df.itertuples():
    Time = getattr(_,'Time')
    HelpfulnessNumerator = getattr(_,'HelpfulnessNumerator')
    itertuples_devide.append(devide(Time,HelpfulnessNumerator))

4 s ± 238 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%timeit df['Time'] /df['HelpfulnessNumerator']

10.2 ms ± 655 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
%timeit list(map(devide,df['Time'],df['HelpfulnessNumerator']))

861 ms ± 22.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%timeit np.vectorize(devide)(df['Time'],df['HelpfulnessNumerator'])

636 ms ± 24.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [29]:
def modified_devide(t,h):
    return t/h

In [30]:
%timeit np.where(df['HelpfulnessNumerator'] == 0 , 0 , modified_devide(df['Time'],df['HelpfulnessNumerator']))

21.4 ms ± 200 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [32]:
%timeit modified_devide(df['Time'],df['HelpfulnessNumerator'])

8.75 ms ± 114 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [28]:
%timeit np.where(df['HelpfulnessNumerator'] == 0 , 0 , df['Time']/df['HelpfulnessNumerator'])

21.6 ms ± 329 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [33]:
%timeit df['Time']/df['HelpfulnessNumerator']

8.83 ms ± 140 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [34]:
from numba import njit

@njit()
def divide(t, h):
    res = np.empty(t.shape)
    for i in range(len(t)):
        if h[i] != 0:
            res[i] = t[i] / h[i]
        else:
            res[i] = 0
    return res

%timeit divide(df['Time'].values, df['HelpfulnessNumerator'].values)

9.66 ms ± 142 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
%load_ext fortranmagic